In [13]:
from sqlalchemy import create_engine
import pandas as pd
import json
import requests
import time
import threading
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor

In [14]:
# 取得 Token , 設置請求參數 (大陸)
url = "https://login.xiaoshouyi.com/auc/oauth2/token"
headers = {
    "Content-Type": "application/x-www-form-urlencoded"
}
data = {
    "grant_type": "password",
    "client_id": "473f31e3a6cb9cafaec3ddf36d20092b",
    "client_secret": "ddfed65f5aeebacd0f54eaa464fcb043",
    "redirect_uri": "https://api-scrm.xiaoshouyi.com/",
    "username": "11021041@twkd.com",
    "password": "Marcellin01oiH88cCI"
}

# 發送 post 請求
response = requests.post(url, headers=headers, data=data)

if response.status_code == 200:
    print(response.json())
else:
    print("請求失敗, 狀態碼:", response.status_code)

content = response.json()
ac_token = content["access_token"]


print(ac_token)

{'access_token': '886ca7301e33b18c18c3c88e3bc429308ae46718598503f203b90ba5c4bd8f10.MjM2OTI2Njk5ODU2MzExMw==0', 'token_type': 'Bearer', 'expires_in': 86399, 'scope': 'all prm crm', 'tenant_id': '2369266998563113', 'login_type': '', 'encryption_key': '', 'client_id': '473f31e3a6cb9cafaec3ddf36d20092b', 'passport_id': '3052172005298442', 'mobile_params': '', 'instance_uri': 'api-scrm.xiaoshouyi.com'}
886ca7301e33b18c18c3c88e3bc429308ae46718598503f203b90ba5c4bd8f10.MjM2OTI2Njk5ODU2MzExMw==0


In [25]:
'''
select from Tasks withdraw 
'''
url_2 = "https://api-scrm.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/x-www-form-urlencoded"
}
queryLocator = ''
Tasks_df1 = pd.DataFrame()

date_to_convert = datetime(2024, 3, 19) 

# date_to_convert = datetime(2024, 8, 12) 
timestamp = int(date_to_convert.timestamp() * 1000)

while True:
    data = {
        "xoql": f'''
        SELECT id,
        name,
        customItem10__c, 
        customItem3__c,
        workflowStageName,
        approvalStatus,
        customItem8__c,
        customItem58__c
        FROM customEntity14__c
        WHERE entityType = '3074631761565155' 
        AND createdBy = '3052177351988706' 
        AND createdAt >= {timestamp}
        ''',
        "batchCount": 2000,
        "queryLocator": queryLocator
    }
    response = requests.post(url_2, headers=headers, data=data)
    crm = response.json()
    data = pd.DataFrame(crm["data"]["records"])
    Tasks_df1 = pd.concat([Tasks_df1, data], ignore_index=True, sort=False)
    
    if not crm['queryLocator']:
        break
    queryLocator = crm['queryLocator']
    
Tasks_df1['customItem8__c']= Tasks_df1['customItem8__c'].astype(str)
Tasks_df1['approvalStatus']= Tasks_df1['approvalStatus'].astype(str)
Tasks_df1 = Tasks_df1.loc[Tasks_df1['customItem8__c'].str.contains("等待")]  
Tasks_df1 = Tasks_df1.loc[~Tasks_df1['customItem8__c'].str.contains("进行中")]  
Tasks_df1 = Tasks_df1.loc[~Tasks_df1['approvalStatus'].str.contains("审批通过")]  

# 電訪狀態 != 已邀約但時間待定 -> 大陸改為判斷"工作主旨"
Tasks_df1['customItem3__c']= Tasks_df1['customItem3__c'].astype(str)
Tasks_df1 = Tasks_df1.loc[~Tasks_df1['customItem3__c'].str.contains("已邀|K大-客户未上线", regex=True)] # 回放

Tasks_df1

,id,name,customItem10__c,customItem3__c,workflowStageName,approvalStatus,customItem8__c,customItem58__c


In [21]:
'''
get procInstId
'''
Tasks_df2 = Tasks_df1[['id']]
base_url = "https://api-scrm.xiaoshouyi.com/rest/data/v2.0/creekflow/history/filter?entityApiKey=customEntity14__c&dataId="

def fetch_data(data_id):
    print(data_id)
    api_url = f"{base_url}{data_id}&stageFlg=false"
    response = requests.get(api_url, headers=headers)
    if response.status_code == 200:
        api_data = response.json()
        if api_data["data"]:
          print(api_data["data"][-1]["procInstId"])
          return json.dumps({ "dataId": data_id, "procInstId": api_data["data"][-1]["procInstId"] })
          # df = pd.DataFrame(api_data["data"][0]["procInstId"], dtype=str)
        else:
            print("Data is empty")
            return
    else:
        print(f"Error accessing API for dataId {data_id}. Status code: {response.status_code}")
        # return pd.DataFrame()
        return
    

# Set the maximum number of threads you want to use
max_threads = 2  # You can adjust this based on your needs


with ThreadPoolExecutor(max_threads) as executor:
    # Use executor.map to asynchronously fetch data for each row in parallel
    dfs = list(executor.map(fetch_data, Tasks_df2['id']))
    # Add a time delay between each thread

json_data = [entry for entry in dfs if entry is not None]
Tasks_df2 = pd.json_normalize([json.loads(entry) for entry in json_data])

3720175876690718
3720175876690719
3720177717826359
3720175876690720
3720177713615654
3720175876690723
3720297003210195
3720175876690732
3720177713615679
3720175877378862
3720177739436851
3720175879771627
3720178044605213
3720175930888978
3720297088275205
3720175932741063
3720178784245226
3720175932741066
3720178851353375
3720175932741081
3720178845062654
3720175933674951
3720178887284217
3720178865395180


In [22]:
'''
withdraw_task
'''

def withdraw_task(row):
    data_id = row['dataId']
    task_id = row['procInstId']
    url_2 = "https://api-scrm.xiaoshouyi.com/rest/data/v2.0/creekflow/task"
    headers = {
        "Authorization": f"Bearer {ac_token}",
        "Content-Type": "application/json"
    }
    data = {
        "data": {
            "action": "withdraw",
            "entityApiKey": "customEntity14__c",
            "dataId": data_id,
            "procInstId": task_id
        }
    }

    response = requests.post(url_2, headers=headers, json=data)
    result = response.json()
    print(f"Response for dataId {data_id}: {result}")

# Assuming data_ids is your DataFrame
data_ids_df = Tasks_df2[['dataId', 'procInstId']]

# Create a thread for each row in the DataFrame
threads = []
for index, row in data_ids_df.iterrows():
    thread = threading.Thread(target=withdraw_task, args=(row,))
    threads.append(thread)
    thread.start()
    time.sleep(0.08)

# Wait for all threads to finish
for thread in threads:
    thread.join()

Response for dataId 3720175876690718: {'code': '200', 'msg': 'OK', 'data': None, 'errorInfo': None}
Response for dataId 3720175876690719: {'code': '200', 'msg': 'OK', 'data': None, 'errorInfo': None}
Response for dataId 3720175876690720: {'code': '200', 'msg': 'OK', 'data': None, 'errorInfo': None}
Response for dataId 3720175876690723: {'code': '200', 'msg': 'OK', 'data': None, 'errorInfo': None}
Response for dataId 3720175876690732: {'code': '200', 'msg': 'OK', 'data': None, 'errorInfo': None}
Response for dataId 3720175877378862: {'code': '200', 'msg': 'OK', 'data': None, 'errorInfo': None}
Response for dataId 3720175879771627: {'code': '200', 'msg': 'OK', 'data': None, 'errorInfo': None}
Response for dataId 3720175930888978: {'code': '200', 'msg': 'OK', 'data': None, 'errorInfo': None}
Response for dataId 3720175932741066: {'code': '200', 'msg': 'OK', 'data': None, 'errorInfo': None}
Response for dataId 3720175932741063: {'code': '200', 'msg': 'OK', 'data': None, 'errorInfo': None}


In [23]:
'''
ask bulk id
'''
url_2 = "https://api-scrm.xiaoshouyi.com/rest/bulk/v2/job"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/json"
}
data = {
     "data": {
        "operation": "delete",####################################################################3
        "object": "customEntity14__c",
        "execOption": ["CHECK_RULE", "CHECK_DUPLICATE"]
    }
    
}
response = requests.post(url_2, headers = headers, json = data)
test = response.json()
bulk_id = test["result"]["id"]
print(bulk_id)

Tasks_df3 = Tasks_df2[['dataId']]
Tasks_df3 = Tasks_df3.rename(columns={'dataId':'id'})
#Tasks_df3 = Tasks_df1[['id']]

e9fa96a8caa871e0ab36de8779d0f0cf6a81dbef7e7e497fa3d89f857d1e4011


In [24]:
'''
delete to crm if dataframe over 5000
'''
batch_size = 5000

# Calculate the number of batches
num_batches = (len(Tasks_df3) + batch_size - 1) // batch_size
# Initialize a list to store responses
all_responses = []
# Initialize an index to keep track of the current batch
current_batch_index = 0

# While there are more batches to process
while current_batch_index < num_batches:
    start_index = current_batch_index * batch_size
    end_index = (current_batch_index + 1) * batch_size
    current_batch_df = Tasks_df3.iloc[start_index:end_index]
    json_data = current_batch_df.to_dict(orient='records')
    
    url_2 = "https://api-scrm.xiaoshouyi.com/rest/bulk/v2/batch"
    headers = {
        "Authorization": f"Bearer {ac_token}",
        "Content-Type":"application/json"
    }
    data = {
        "data": {
            "jobId": f"{bulk_id}",
            "datas": json_data
        }
    }
    response = requests.post(url_2, headers=headers, json=data)
    all_responses.append(response)
    current_batch_index += 1
for response in all_responses:
    pass